# Back Bay National Wildlife Refuge


> Back Bay National Wildlife Refuge is located in the southeastern corner of the City of Virginia Beach. The refuge was established in 1938 to protect and provide habitat for migrating and wintering waterfowl. Diverse habitats, including beachfront, freshwater marsh, dunes, shrub-scrub and upland forest are home to hundreds of species of birds, reptiles, amphibians, mammals and fish.

![BNWR](https://www.fws.gov/sites/default/files/styles/banner_image_xl/public/banner_images/2020-09/waterfowl%20%28tundras%29.jpg?h=0c8d0f81&itok=NcZlpD27)


To get introduced to the park and its history, please view the following interactive story map.

[BBNWR History and Introduction](https://storymaps.arcgis.com/stories/960d9db38cca4f3d8d38111119b9874f)

Additionally, here is some drone footage of the park for a better look at the geography and ecology of the area.

[BBNWR Drone Footage](https://www.youtube.com/watch?v=NlW330aBTCc)


In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sb

In [ ]:
bbnwr = pd.read_csv("https://github.com/UM-Data-Science-101/lab-10/raw/refs/heads/main/BKB_WaterQualityData_2020084.csv")
bbnwr.columns

In [ ]:
bbnwr.head()

In [ ]:
bbnwr["Site_Id"].replace('d', 'D', inplace = True)
bbnwr["Site_Id"].value_counts(normalize = True)

In [ ]:
dissox = bbnwr["Dissolved Oxygen (mg/L)"].dropna()


## Testing Hypotheses

In the previous lab and homework, we investigated the behavior of the sample mean as our sample size grew larger. As we would expect from the Central Limit Theorem, as the sample size got bigger and bigger, the sample mean behaved more and more like a Gaussian distribution, centered at the population mean and with a standard deviation equal to the standard error.

Let's apply those ideas to the original BNWR data, treating it as a random sample from the population of possible measurements that could have occurred (perhaps best thought of as a data generating process).


Using `dissox` as the sample, compute the SEM as the ratio of the sample standard deviation to the square root of the sample size.

<details>

```
stdx = dissox.std()
n = len(dissox)
sem = stdx / np.sqrt(n)

sem
```
</details>

Now we have estimated the standard error of the mean. Consider the hypothesis that the true population average dissolved oxygen is 7 mg/L. If this was the truth, what range of values would include the sample mean approximately 95% of the time?

<details>

`(7 - 2 * sem, 7 + 2 * sem)`

</details>

Compute the sample mean. Is it in that iterval. At the 5% $\alpha$ (confidence) level, would you reject the null hypothesis that the population average dissolved oxygen in 7 in favor of the alternative that it is not 7?

<details>

`dissox.mean()`

We see the sample means falls out side this range, so we would reject the null hypothesis at the 5% level.
    
</details>



Now do a hypothesis test of the hypothesis that the average population "Salinity (ppt)" is 0.8 against the alternative that it is not equal to 0.8. Use a 5% $\alpha$ level.

<details>

```
salinity = bbnwr["Salinity (ppt)"].dropna()
salinity_sem = salinity.std() / np.sqrt(len(salinity))
(0.8 - 2 * salinity_sem, 0.8 + 2 * salinity_sem)
    
salinity.mean()
```

We see that the sample mean is  outside of this range, so we reject the null hypothesis at the 5% level.
    
</details>



We could also compute a "Z test" by subtracting the hypothesized value form the sample mean and dividing by the SEM. The result should be approximately a Gaussian distribution with mean zero and variance one if the hypothesis is true.

Compute the Z test for Salinity.

How many standard deviations are we away from the hypothesized population mean? If the hypothesis that the population mean was 0.8 was true what is the approximate probability of observing a sample mean that is as far or farther than the observed sample mean?

<details>

`(salinity.mean() - 0.8) / sem`

1.3 standard deviations away. The emprical rule doesn't quite fit 1.2 standard deviations, but we can roughly interpolate. This is somewhere between 32% and 5%, so probably close to 20% of sample means will be 1.3 standard deviations away from the mean. A fairly common event.

</details>

## From Testing to p-values

Above, we performed two hypothesis tests. In the first, we tested the null hypothesis that mean `dissox` value in the population was equal to 7 mg/L against the alternative that it was not 7 mg/L. In the second test, we tested the null hypothesis the average salinity in the population was 0.8, again against the two-sided alternative.

We recreate these analyses here. In each case, we will set the $\alpha = 0.05$ level (a typical Type I error probability).

In [ ]:
dissox = bbnwr["Dissolved Oxygen (mg/L)"].dropna()
dissox_sem = dissox.std() / np.sqrt(len(dissox))

dissox_z = (dissox.mean() - 7) / dissox_sem
dissox_z

Under the null hypothesis the statistic above should fall with within 2 standard deviations of 0 with 95% probability. As we observe a statistic outside of this range, we reject the null hypothesis.

In [ ]:
salinity = bbnwr["Salinity (ppt)"].dropna()
salinity_sem = salinity.std() / np.sqrt(len(salinity))
salinity_z = (salinity.mean() - 0.8)/ salinity_sem
salinity_z

Likewise for salinity.

Why whould we use $\alpha = 0.05$? This is a common tolerance for Type I error, but certainly not the only value.

Instead, we could ask, "What is is the smallest rejection region that would include the observed test statistic?" Since we are thinking about two tailed tests, this is equivalent to asking, "What is the probabiliy of observing a Gaussian random variable exceeding $|\bar X - c| / \text{SEM}$ or less than $-|\bar X - c| / \text{SEM})$?"

In [ ]:
from scipy.stats import norm

dissox_z_abs = np.abs(dissox_z)
norm.cdf(- dissox_z_abs) + (1 - norm.cdf(dissox_z_abs))

In [ ]:
## or by the symmetry of the Gaussian distribution
2 * norm.cdf(- dissox_z_abs)

For any $\alpha$ a person selects, we can decide if we reject the null hypothesis if the $p$-value is less than $\alpha$. With a $p$-value of $3.7 \times 10^{-8}$, we would reject this null hypothesis with almost any typical significance/$\alpha$ level.

Repeat this computation for the salinity test. Would we reject this hypothesis at the $\alpha = 0.001$ level?


<details>

salinity_z_abs = np.abs(salinity_z)

norm.cdf(- salinity_z_abs) + (1 - norm.cdf(salinity_z_abs))


</details>

<details>
2 * norm.cdf(- dissox_z_abs)
</details>

## Other Statistics

Many other statistics have approximate Gaussian distributions in large samples with the following standard errors:

* Difference of means ($\bar X - \bar Y$) with sample sizes $n$ and $m$: $\sqrt{S_x^2/n + S_y^2/m}$
* Proportions ($\hat p$): $\sqrt{\hat p (1 - \hat p) / n}$
* Correlation: $1/\sqrt{n}$



Let's test the hypothesis that in the population of all tests, 1/3 of them come from the Bay against the two sided alternative that the proportion is some other value.

Create a new column that indicates if a reading came from the "Bay" or a numbered site.



<details>

```
bbnwr["is_bay"] = bbnwr["Site_Id"] == "Bay"
p_hat = bbnwr["is_bay"].mean()
p_hat
```

</details>


Calculate the estimated standard error of this statistic.

<details>

```
se_p_hat = np.sqrt(p_hat * (1 - p_hat) / bbnwr["is_bay"].count())

```

</details>

Using the sample proportion and the estimated standard error for that proportion, apply the concept of a Z-test to this hypothesis test. At the 5% level, would you reject the hypothesis that 1/3 of the observations would come from the Bay in the population of all observations?


<details>

```
(p_hat - 1/3) / se_p_hat
```

</details>

At the 5% level, would we reject the hypothesis that 1/3 of the population measurements occur at the Bay?

# **Hypothesis Testing Between Hour_Minute and Temperature**

Let's do one last test to practice this technique. The following creates a new column that gives time in fractional hours (e.g., 9:30 becomes 9.5).

Create a table with just the non-missing entries for `hour_minute` and `AirTemp (C)`. Create a scatter plot of these two columns.



In [ ]:
bbnwr["dt"] = pd.to_datetime(bbnwr["Read_Date"] + " " + bbnwr["Time (24:00)"], errors = 'coerce')
bbnwr["hour_minute"] = bbnwr["dt"].dt.hour + bbnwr["dt"].dt.minute/60

<details>

```
time_temp = bbnwr[["hour_minute", "AirTemp (C)"]].dropna()
sb.scatterplot(data = time_temp, x = "hour_minute", y = "AirTemp (C)")
```
</details>

Let's test the hypothesis that the correlation between the "hour_minute" and air temperature is zero or less against the alternative that is is more than zero.

Compute the standard error for the correlation coefficient statistic for this setting and perform a Z-test selecting a rejection region that will ensure a 0.16 probability of rejecting a true null hypothesis. What do you conclude?



<details>

For this test, after subtracting the hypothesized value of 0 and dividing by the SE, the result should be approximately distributed as N(0,1). So to control Type I error at the 0.16 level, we would reject for a test statistic greater than 1.

```
(n, _) = time_temp.shape

corr_se = 1/np.sqrt(n)

test_stat = (time_temp.corr()["hour_minute"]["AirTemp (C)"] - 0) / corr_se
test_stat
```

With a value exceeding 1, we would reject this null hypothesis
</details>


## Extra Section: Quick and Easy Regression Analysis with `statsmodels`
Let's try something a little more advanced! We'll use `statsmodels` to fit a linear regression model
and see how it handles the heavy lifting for us. Here, we aim to analyze how one of our variables - Water Temp (C) relates to `Dissolved Oxygen (mg/L)`.

Ready for some stats magic?


In [ ]:

import statsmodels.api as sm

bbnwr_cleaned = bbnwr[['Dissolved Oxygen (mg/L)', 'Water Temp (?C)']].dropna()
X = bbnwr_cleaned['Water Temp (?C)']  # Independent variable
y = bbnwr_cleaned['Dissolved Oxygen (mg/L)']  # Dependent variable

# Adding a constant term for intercept in the model
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Display the model summary
print(model.summary())



### Interpreting the Results
Check out the table above! `statsmodels` has provided us with an abundance of information:
- **Coefficient**: This tells us the relationship between Temperature and Dissolved Oxygen levels. It's -0.1566 here, so the relation is negative. Hence as water temperature increases, the dissolved oxygen decreases, which corroborates with the known scientific fact that **warmer water holds less dissolved oxygen.**
- **P-value**: Is the relationship statistically significant? P-value here is 0.000, hence it is significant.
- **R-squared**: How well does our model explain the variability in Dissolved Oxygen levels? R-sq here is 0.263, so doesn't explain much but only to some extent.

Using `statsmodels` is like having a personal statistician; it automates calculations, so you don’t have to sweat the details.